# <center>Classifier 2: topology</center>
---

In [8]:
import pickle
import pandas as pd
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import RidgeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
pd.set_option('display.max_columns', None)

First, load in the training sets generated in `dataset.ipynb`.

In [9]:
X = pd.read_csv('../data/X_train_c2.csv')
y = pd.read_csv('../data/y_train_c2.csv')

### Inputs: structural/compositional features **and band gap**

In [10]:
X.head(5)

,gap pbe,minimum CN_VoronoiNN,maximum CN_VoronoiNN,range CN_VoronoiNN,mean absolute deviation in relative bond length,minimum neighbor distance variation,maximum neighbor distance variation,mean ordering parameter shell 1,minimum local difference in Number,maximum local difference in Number,minimum local difference in MendeleevNumber,maximum local difference in MendeleevNumber,minimum local difference in MeltingT,maximum local difference in MeltingT,minimum local difference in CovalentRadius,maximum local difference in CovalentRadius,minimum local difference in NsValence,maximum local difference in NsValence,minimum local difference in NpValence,maximum local difference in NpValence,minimum local difference in NdValence,maximum local difference in NdValence,minimum local difference in NfValence,maximum local difference in NfValence,minimum local difference in NValence,maximum local difference in NValence,minimum local difference in NpUnfilled,maximum local difference in NpUnfilled,minimum local difference in NdUnfilled,maximum local difference in NdUnfilled,minimum local difference in NfUnfilled,maximum local difference in NfUnfilled,minimum local difference in GSvolume_pa,maximum local difference in GSvolume_pa,minimum local difference in GSbandgap,maximum local difference in GSbandgap,minimum local difference in GSmagmom,maximum local difference in GSmagmom,minimum local difference in SpaceGroupNumber,maximum local difference in SpaceGroupNumber,0-norm,2-norm,MagpieData minimum Number,MagpieData maximum Number,MagpieData mean MendeleevNumber,MagpieData mode MendeleevNumber,MagpieData minimum MeltingT,MagpieData maximum MeltingT,MagpieData maximum NsValence,MagpieData minimum NpValence,MagpieData minimum NdValence,MagpieData mean NdValence,MagpieData minimum NfValence,MagpieData mode NfValence,MagpieData minimum NValence,MagpieData mode NsUnfilled,MagpieData minimum NpUnfilled,MagpieData mode NpUnfilled,MagpieData minimum NdUnfilled,MagpieData mode NdUnfilled,MagpieData minimum NfUnfilled,MagpieData mode NfUnfilled,MagpieData minimum NUnfilled,MagpieData mode NUnfilled,MagpieData minimum GSvolume_pa,MagpieData mean GSvolume_pa,MagpieData mode GSvolume_pa,MagpieData minimum GSbandgap,MagpieData minimum GSmagmom,MagpieData mode GSmagmom,MagpieData maximum SpaceGroupNumber,compound possible
0,0.3004,4.211860,10.798291,6.586431,0.100886,0.032640,0.233917,0.651250,7.000000,40.876733,2.723842,22.925931,110.748155,1300.560225,20.067270,75.612589,0.000000,0.999837,0.679684,3.981769,0.000000,9.941301,0.000000,0.0,1.000000,6.970651,0.678214,3.012017,0.0,0.0,0.000000,0.0,2.030451,15.143466,0.000000,1.625000,0.0,0.0,10.000000,212.221072,4,0.661438,8.0,49.0,82.125,87.0,54.80,1357.77,2.0,0.0,0.0,2.5,0.0,0.0,5.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,2.0,9.105,12.928155,9.105,0.0,0.0,0.0,225.0,True
1,0.6904,6.049687,10.551066,4.501380,0.060289,0.010302,0.158696,0.637178,24.978893,41.832903,7.012532,16.975471,304.987031,537.842166,46.266300,77.744199,0.000000,0.000000,1.894115,3.992992,6.107502,9.964959,0.000000,0.0,4.213387,7.974936,1.221500,2.008355,0.0,0.0,0.000000,0.0,10.130261,24.136597,0.000000,0.000000,0.0,0.0,96.611064,181.547969,3,0.663325,8.0,50.0,82.200,87.0,54.80,594.22,2.0,0.0,0.0,4.0,0.0,0.0,6.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,9.105,16.019000,9.105,0.0,0.0,0.0,194.0,True
2,3.6626,6.207261,9.503291,3.296030,0.057181,0.018408,0.097328,0.752606,8.943151,50.813181,64.228361,69.048924,226.860713,1144.415237,104.434952,134.673260,0.016984,0.776245,3.049196,3.932066,0.000000,0.000000,0.041840,3.0,1.050951,3.867281,1.524598,1.966033,0.0,0.0,0.153412,11.0,26.754469,49.310445,0.000000,0.000000,0.0,0.0,165.906996,179.503425,3,0.612372,8.0,59.0,48.500,87.0,54.80,1204.00,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,2.0,9.105,31.747917,9.105,0.0,0.0,0.0,229.0,True
3,1.9258,6.002047,7.556327,1.554279,0.025872,0.000164,0.069238,0.882672,20.328464,34.003923,53.996418,76.865700,109.001283,1276.021001,40.642514,68.992154,0.000171,0.88

### Output: trivial or non-trivial

In [11]:
y.value_counts()

topology
1           9408
0           6975
dtype: int64

### Training

In [12]:
def classify(model_type, par_dict, X_train, y_train):
    '''
    The skeleton used for training a classifier.
    INPUTS: 
        model_type -- A scikit-learn estimator object.
        par_dict -- The dict of parameters used for CV.
        X_train -- the input of the training set.
        y_train -- the ouput of the training set.
    OUTPUT:
        A trained scikit-learn model.
    '''
    rscv = RandomizedSearchCV(
            estimator = model_type, 
            param_distributions = par_dict, 
            scoring="neg_mean_squared_error",
            verbose = 2, 
            random_state = 42, 
            n_jobs = -1
    )
    rscv.fit(X_train, y_train.values.ravel())
    print(rscv.best_params_)
    model = rscv.best_estimator_
    model.fit(X_train, y_train.values.ravel())
    return model

Let's consider both linear and non-linear models.

In [13]:
choices = [
    [SVC(), {"C": np.logspace(-1, 5, 10)}],
    [RandomForestClassifier(), {'n_estimators': [int(x) for x in np.linspace(200, 1000, 5)] + [1500, 2000], 'max_depth': [int(x) for x in np.linspace(5, 55, 11)] + [None], 'max_features': ['auto', 'sqrt', 'log2'], 'criterion': ['gini', 'entropy'], 'min_samples_split': [int(x) for x in np.linspace(2, 10, 9)], 'min_impurity_decrease': [0.0, 0.05, 0.1]}],
    [RidgeClassifier(), {"alpha": np.logspace(-6, 3, 10), "class_weight": [None, "balanced"]}],
    [MLPClassifier(), {'hidden_layer_sizes':[(10,10,10),(15,15,15),(20,20)], "alpha": [0.00001,0.0001,0.001,0.01], "activation": ["identity","logistic", "tanh", "relu"]}],
    [LogisticRegression(), {"C": [0.001, 0.01, 0.1, 1, 10, 100], "class_weight": [None, 'balanced']}],
    [KNeighborsClassifier(), {"n_neighbors": list(range(1,20,1)), "weights": ["uniform", "distance"], "p": [1, 2, 3]}]
]

The next cell trains all models and writes them to `pickle` files in the `jar`. Of course, with a dataset of this size, running it locally would take forever. Hence, I trained all of the models on HPC clusters.

In [ ]:
for choice in choices:
    filename = str(choice[0])[:-2] + '_trained.p'
    trained_model = classify(choice[0], choice[1], X, y)
    pickle.dump(trained_model, open('../jar/classifier2/' + filename, 'wb'))